### Домашнее задание

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
%matplotlib inline

In [2]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=24)

In [5]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

gender = Pipeline([
                ('selector', FeatureSelector(column='Gender')),
                ('ohe', OHEEncoder(key='Gender'))
            ])
gender.fit(X_train)
gender.transform(X_test).head(3)

,Gender_Female,Gender_Male
5821,1,0
2516,1,0
4282,1,0


In [8]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [9]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [10]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [11]:
%%time
pipeline.fit(X_train, y_train)

Wall time: 758 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [12]:
preds_RandomForestCr = pipeline.predict_proba(X_test)[:, 1]

In [13]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_RandomForestCr)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.360000, F-Score=0.617, Precision=0.652, Recall=0.585


In [14]:
result = []
result.append(['RandomForestClassifier', precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds_RandomForestCr)])

In [15]:
matrix = []
cnf_RandomForestCr = confusion_matrix(y_test, preds_RandomForestCr>=thresholds[ix])
matrix.append([cnf_RandomForestCr[0,0],cnf_RandomForestCr[0,1],cnf_RandomForestCr[1,0],cnf_RandomForestCr[1,1]])

Произведём подбор параметров

In [16]:
from sklearn.model_selection import GridSearchCV

params={'classifier__max_features':[0.3, 0.5, 0.7],
        'classifier__min_samples_leaf':[1, 2, 3],
        'classifier__max_depth':[None]
        }

In [17]:
grid = GridSearchCV(pipeline,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': None,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 2}

In [18]:
pipeline_rfc_optim = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth=search.best_params_['classifier__max_depth'],
                                          max_features=search.best_params_['classifier__max_features'], 
                                          min_samples_leaf=search.best_params_[ 'classifier__min_samples_leaf'],
                                          random_state=42)),
])
pipeline_rfc_optim.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [19]:
preds_RandomForestCr_optim = pipeline_rfc_optim.predict_proba(X_test)[:, 1]

In [20]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_RandomForestCr_optim)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.313108, F-Score=0.634, Precision=0.621, Recall=0.648


In [21]:
result.append(['RandomForestClassifier optim', precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds_RandomForestCr_optim)])

In [22]:
cnf_RandomForestCr_optim = confusion_matrix(y_test, preds_RandomForestCr_optim>=thresholds[ix])
matrix.append([cnf_RandomForestCr_optim[0,0],cnf_RandomForestCr_optim[0,1],
               cnf_RandomForestCr_optim[1,0],cnf_RandomForestCr_optim[1,1]])

### Бустинг

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

In [24]:
pipeline_gbc = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42))])

In [25]:
%%time
pipeline_gbc.fit(X_train, y_train)

Wall time: 1.03 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [26]:
preds_gbc = pipeline_gbc.predict_proba(X_test)[:, 1]

In [27]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_gbc)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.312373, F-Score=0.627, Precision=0.633, Recall=0.621


In [28]:
result.append(['GradientBoostingClassifier', precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds_gbc)])

In [29]:
cnf_gbc = confusion_matrix(y_test, preds_gbc>=thresholds[ix])
matrix.append([cnf_gbc[0,0],cnf_gbc[0,1],
               cnf_gbc[1,0],cnf_gbc[1,1]])

Подбор параметров

In [30]:
params={'classifier__n_estimators':[50, 100, 150],
       'classifier__min_samples_leaf':[1,2,3], 
       'classifier__max_depth':[3,4,5]
       }

In [31]:
grid = GridSearchCV(pipeline_gbc,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_depth': 3,
 'classifier__min_samples_leaf': 1,
 'classifier__n_estimators': 150}

In [32]:
pipeline_gbc_opmim = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(max_depth = search.best_params_['classifier__max_depth'], 
                                              min_samples_leaf = search.best_params_['classifier__min_samples_leaf'], 
                                              n_estimators= search.best_params_['classifier__n_estimators'], 
                                              random_state = 42)),
])

In [33]:
%%time
pipeline_gbc_opmim.fit(X_train, y_train)

Wall time: 1.62 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [34]:
preds_gbc_optim = pipeline_gbc_opmim.predict_proba(X_test)[:, 1]

In [35]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_gbc_optim)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.343216, F-Score=0.630, Precision=0.673, Recall=0.593


In [36]:
result.append(['GradientBoostingClassifier optim', precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds_gbc_optim)])


In [37]:
cnf_gbc_optim = confusion_matrix(y_test, preds_gbc_optim>=thresholds[ix])
matrix.append([cnf_gbc_optim[0,0],cnf_gbc_optim[0,1],
               cnf_gbc_optim[1,0],cnf_gbc_optim[1,1]])

### Логистическая регрессия

In [38]:
pipeline_lr = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [39]:
%time
pipeline_lr.fit(X_train, y_train)

Wall time: 0 ns


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [40]:
preds_LogReg = pipeline_lr.predict_proba(X_test)[:, 1]

In [41]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_LogReg)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore[:-5])
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.257403, F-Score=0.502, Precision=0.434, Recall=0.595


<ipython-input-41-114c2e57a92f>:3: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


In [42]:
result.append(['LogisticRegression', precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds_LogReg)])

In [43]:
cnf_LogReg = confusion_matrix(y_test, preds_LogReg>=thresholds[ix])
matrix.append([cnf_LogReg[0,0],cnf_LogReg[0,1],
               cnf_LogReg[1,0],cnf_LogReg[1,1]])

Подбор параметров

In [44]:
params={'classifier__class_weight':['balanced',{1:3, 0:1},{1:4, 0:1}]}

In [45]:
grid = GridSearchCV(pipeline_lr,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__class_weight': {1: 3, 0: 1}}

In [46]:
pipeline_lr_optim = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(class_weight = search.best_params_['classifier__class_weight'], 
                                              random_state = 42)),
])

In [47]:
%time
pipeline_lr_optim.fit(X_train, y_train)

Wall time: 0 ns


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [48]:
preds_lr_optim = pipeline_lr_optim.predict_proba(X_test)[:, 1]

In [49]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_lr_optim)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore[:-5])
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.525468, F-Score=0.502, Precision=0.445, Recall=0.575


<ipython-input-49-a96f5a3a9e4e>:3: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


In [50]:
preds_lr_optim_pred = pipeline_lr_optim.predict(X_test)

In [51]:
result.append(['LogisticRegression optim', precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, preds_lr_optim)])


In [52]:
cnf_lr_optim = confusion_matrix(y_test, preds_lr_optim>=thresholds[ix])
matrix.append([cnf_lr_optim[0,0],cnf_lr_optim[0,1],
               cnf_lr_optim[1,0],cnf_lr_optim[1,1]])

Соберём все результаты в единый датасет

In [53]:
result_pr = pd.DataFrame(result, columns = ['model','prec', 'rec', 'f1', 'roc_auc'])

In [54]:
result_mx = pd.DataFrame(matrix, columns = ['TN', 'FP', 'FN','TP'])

In [55]:
res = pd.concat([result_pr, result_mx], axis=1)
res

,model,prec,rec,f1,roc_auc,TN,FP,FN,TP
0,RandomForestClassifier,0.651982,0.584980,0.616667,0.852192,1836,158,210,296
1,RandomForestClassifier optim,0.621212,0.648221,0.634429,0.863200,1794,200,178,328
2,GradientBoostingClassifier,0.633065,0.620553,0.626747,0.865242,1812,182,192,314
3,GradientBoostingClassifier optim,0.672646,0.592885,0.630252,0.865505,1848,146,206,300
4,LogisticRegression,0.434343,0.594862,0.502085,0.773559,1602,392,205,301
5,LogisticRegression optim,0.444954,0.575099,0.501724,0.776277,1631,363,215,291


### Экономические показатели.

In [56]:
income = 2                 # доход с каждого удержанного
expenses = 1               # затраты на удержание
ROS_one = income/expenses  # рентабельность одного клиента

In [57]:
res['income'] = res['TP']*income                     # Суммарный доход
res['expenses'] = (res['FP'] + res['TP'])*expenses   # Суммарные затраты 
res['profit'] = res['income']- res['expenses']       # Итоговая прибыль
res['ROS_project'] = ROS_one*res['prec']             # Рентабельность всего проекта

In [58]:
feature = ['model', 'income','expenses','profit','ROS_project']

In [59]:
res[feature]

,model,income,expenses,profit,ROS_project
0,RandomForestClassifier,592,454,138,1.303965
1,RandomForestClassifier optim,656,528,128,1.242424
2,GradientBoostingClassifier,628,496,132,1.266129
3,GradientBoostingClassifier optim,600,446,154,1.345291
4,LogisticRegression,602,693,-91,0.868687
5,LogisticRegression optim,582,654,-72,0.889908


In [60]:
print('Лучшый вариант - ',list(res.loc[res.profit==res.profit.max()]['model'])[0])

Лучшый вариант -  GradientBoostingClassifier optim
